In [187]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib as plt

In [52]:
# Set simulation parameters
T = 1000  # number of time steps
sigma = 0.02
n = 10000
# Initialize arrayP
P_0 = 100
P_sum = []
for i in range(n):
    r = np.random.normal(0, sigma, size = T)
    P = P_0 + np.cumsum(r)
    P_sum.append(P[-1])


# Calculate expected sd
dd = sigma * np.sqrt(T)


# Print mean and standard deviation of P
print("Mean of P:", np.mean(P_sum))
print("Expected Mean of P:", 100)
print("Standard deviation of P:", np.std(P_sum))
print("Expected Standard deviation of P:", dd)

Mean of P: 100.0106736464363
Expected Mean of P: 100
Standard deviation of P: 0.6317597302656832
Expected Standard deviation of P: 0.6324555320336759


In [185]:
# Set simulation parameters
T = 1000  # number of time steps
sigma = 0.02
n = 10000
# Initialize arrayP
P_0 = 100
P_sum = []
exp_std = []
p_2 = []

for i in range(n):
    r = np.random.normal(0, sigma, size = T)
    P = P_0 * np.cumprod(1+r)
    P_sum.append(P[-1])
    p_2.append(P[-2])

    

# Print mean and standard deviation of P
print("Mean of P:", np.mean(P_sum))
print("Standard deviation of P:", np.std(P_sum))
exp_sd = np.mean(np.std(p_2))
print("Expected Mean of P:", 100)
print("Expected Standard deviation of P:", exp_sd)

Mean of P: 99.5979556116782
Standard deviation of P: 70.1740386041003
Expected Mean of P: 100
Expected Standard deviation of P: 70.0908271945894


In [186]:
# Set simulation parameters
T = 1000  # number of time steps
sigma = 0.02
n = 10000
# Initialize arrayP
P_0 = 100
P_sum = []
for i in range(n):
    r = np.random.normal(0, sigma, size = T)
    P = P_0 * np.exp(np.cumsum(r))
    P_sum.append(P[-1])
# Print mean and standard deviation of P
print("Mean of P:", np.mean(P_sum))
print("Standard deviation of P:", np.std(P_sum))
# Expected value
mu = (sigma**2/2)
PE = P_0 * np.exp(mu*T)
SE = P_0 * np.sqrt((np.exp(sigma**2 * T) - 1) * np.exp(2 * mu * T))
print("Expected Mean of P:", PE)
print("Expected Standard deviation of P:", SE)

Mean of P: 122.55662658790327
Standard deviation of P: 86.57656685285664
Expected Mean of P: 122.14027581601698
Expected Standard deviation of P: 85.65723733877934


In [58]:
def return_calculate(prices, method='DISCRETE', date_column='Date'):
    vars = prices.columns
    n_vars = len(vars)
    vars = [v for v in vars if v != date_column]
    if n_vars == len(vars):
        raise ValueError(f'{date_column} not in DataFrame {vars}')
    n_vars = len(vars)
    p = prices[vars].to_numpy()
    n, m = p.shape
    p2 = np.empty((n - 1, m))
    for i in range(n - 1):
        p2[i, :] = p[i + 1, :] / p[i, :]
    if method.upper() == 'DISCRETE':
        p2 -= 1.0
    elif method.upper() == 'LOG':
        p2 = np.log(p2)
    else:
        raise ValueError(f'method {method} must be in ("LOG", "DISCRETE")')
    dates = prices.iloc[1:, prices.columns.get_loc(date_column)]
    out = pd.DataFrame({date_column: dates})
    for i in range(n_vars):
        out[vars[i]] = p2[:, i]
    return out

In [189]:
prices = pd.read_csv('./DailyPrices.csv')
prices.head()

,Date,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,2/14/2022 0:00,432.011322,167.863144,292.261475,155.167007,291.920013,135.526001,135.300003,217.699997,242.443298,...,197.263107,64.592575,46.290192,151.118790,58.443172,377.068665,247.324020,48.182598,60.352272,66.789505
1,2/15/2022 0:00,438.978333,171.749573,297.680664,156.510498,307.476654,136.608505,136.425507,221.000000,264.702484,...,199.789520,64.328896,46.502743,159.029022,58.653572,372.440185,255.490829,49.446892,62.076630,67.673614
2,2/16/2022 0:00,439.470337,171.511032,297.333191,158.100494,307.796661,137.738007,137.487503,216.539993,264.862305,...,201.139511,64.172638,46.770744,165.211960,59.198696,377.000458,256.349976,50.203476,61.990410,65.915215
3,2/17/2022 0:00,430.082642,167.863144,288.626679,154.652496,292.116669,132.539002,132.308502,207.710007,244.841064,...,194.109894,64.514450,47.473091,157.448944,58.615318,378.822571,248.439911,48.600708,59.921181,64.019295
4,2/18/2022 0:00,427.297852,166.292648,285.846893,152.601502,285.660004,130.403000,130.467499,206.160004,236.199127,...,193.984528,64.455841,47.815025,157.380234,58.385788,376.571686,245.042847,48.381695,59.700851,64.981995


In [190]:
arith_return= return_calculate(prices, method='DISCRETE', date_column='Date')
meta = arith_return['META']
meta_0 = meta - np.mean(meta)
meta_0


/var/folders/p2/bvr89gks5s5ggwkwcjkf1hyc0000gn/T/ipykernel_29920/3205168840.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:, i]


1      0.015175
2     -0.020165
3     -0.040761
4     -0.007446
5     -0.019774
         ...   
244   -0.011850
245   -0.002503
246    0.029899
247   -0.042725
248   -0.030022
Name: META, Length: 248, dtype: float64

In [97]:

from scipy.stats import norm, t


# Define the dataset
data = meta_0

# Define the significance level
alpha = 0.05

# Using a normal distribution
mean = np.mean(data)
std = np.std(data)
VaR_norm = norm.ppf(alpha) * std * np.sqrt(1)
print("VaR using normal distribution: ", VaR_norm)

# Using a normal distribution with an Exponentially Weighted variance (λ = 0.94)
variance = meta.ewm(alpha=0.06).var().iloc[-1]
std_dev_ewm = np.sqrt(variance)
z_score_ewm = stats.norm.ppf(alpha)
var_ewm = (std_dev_ewm * z_score_ewm)
print(f"VaR using normal distribution with EWM variance: {var_ewm:.5f}")

# Using a MLE fitted T distribution
df, loc, scale = t.fit(data)
VaR_t = t.ppf(alpha, df, loc, scale) * np.sqrt(df / (df - 2))
print("VaR using T distribution: ", VaR_t)

# 4. VaR using fitted AR(1) model
from statsmodels.tsa.api import AutoReg

model = AutoReg(meta, lags=1)
results = model.fit()
rho = results.params[1]
var_ar1 = -(rho * meta.mean() + np.sqrt(1 - rho ** 2) * meta.std() * stats.norm.ppf(1-alpha))
print(f"VaR using fitted AR(1) model: {var_ar1:.5f}")

# Using a Historic Simulation
sorted_returns = np.sort(data)
VaR_hs = np.percentile(sorted_returns, alpha*100)
print("VaR using historic simulation: ", VaR_hs)

VaR using normal distribution:  -0.06546917484881122
VaR using normal distribution with EWM variance: -0.09019
VaR using T distribution:  -0.08222424439916937
VaR using fitted AR(1) model: -0.06560
VaR using historic simulation:  -0.0546200790823787


In [191]:
portfolio = pd.read_csv('Portfolio.csv')
prices = pd.read_csv('DailyPrices.csv')

# Get the list of stocks in each portfolio
portfolio_A = portfolio[portfolio['Portfolio'] == 'A']['Stock'].tolist()
portfolio_B = portfolio[portfolio['Portfolio'] == 'B']['Stock'].tolist()
portfolio_C = portfolio[portfolio['Portfolio'] == 'C']['Stock'].tolist()

# Get the number of holdings for each stock in each portfolio
holdings_A = portfolio[portfolio['Portfolio'] == 'A']['Holding'].tolist()
holdings_B = portfolio[portfolio['Portfolio'] == 'B']['Holding'].tolist()
holdings_C = portfolio[portfolio['Portfolio'] == 'C']['Holding'].tolist()

# Get the daily prices for the stocks in each portfolio
portfolio_A_prices = prices[portfolio_A].values
portfolio_B_prices = prices[portfolio_B].values
portfolio_C_prices = prices[portfolio_C].values

# Calculate the daily returns for each portfolio
portfolio_A_returns = np.diff(np.log(portfolio_A_prices), axis=0)
portfolio_B_returns = np.diff(np.log(portfolio_B_prices), axis=0)
portfolio_C_returns = np.diff(np.log(portfolio_C_prices), axis=0)

# Calculate the covariance matrix for each portfolio using an exponentially weighted covariance with lambda = 0.94
cov_A = np.cov(portfolio_A_returns, rowvar=False, aweights=np.power(0.94, np.arange(len(portfolio_A_returns)-1, -1, -1)))
cov_B = np.cov(portfolio_B_returns, rowvar=False, aweights=np.power(0.94, np.arange(len(portfolio_B_returns)-1, -1, -1)))
cov_C = np.cov(portfolio_C_returns, rowvar=False, aweights=np.power(0.94, np.arange(len(portfolio_C_returns)-1, -1, -1)))

# Calculate the total covariance matrix using an exponentially weighted covariance with lambda = 0.94
total_returns = np.concatenate((portfolio_A_returns, portfolio_B_returns, portfolio_C_returns), axis=1)
total_cov = np.cov(total_returns, rowvar=False, aweights=np.power(0.94, np.arange(len(total_returns)-1, -1, -1)))

# Calculate the portfolio values for each portfolio
portfolio_A_values = portfolio_A_prices[-1,:] * holdings_A
portfolio_B_values = portfolio_B_prices[-1,:] * holdings_B
portfolio_C_values = portfolio_C_prices[-1,:] * holdings_C

# Calculate the total portfolio value
total_portfolio_value = np.sum(portfolio_A_values) + np.sum(portfolio_B_values) + np.sum(portfolio_C_values)

# Calculate the VaR for each portfolio and the total VaR
confidence_level = 0.95
z_score = norm.ppf(confidence_level)
portfolio_A_var = z_score * np.sqrt(np.dot(portfolio_A_values, np.dot(cov_A, portfolio_A_values)))
portfolio_B_var = z_score * np.sqrt(np.dot(portfolio_B_values, np.dot(cov_B, portfolio_B_values)))
portfolio_C_var = z_score * np.sqrt(np.dot(portfolio_C_values, np.dot(cov_C, portfolio_C_values)))
total_var = z_score * np.sqrt(np.dot(np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)), np.dot(total_cov, np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)))))

portfolio_A_var, portfolio_B_var, portfolio_C_var, total_var
 # Print the results
print(f"Portfolio A VaR: ${portfolio_A_var:.2f}")
print(f"Portfolio B VaR: ${portfolio_B_var:.2f}")
print(f"Portfolio C VaR: ${portfolio_C_var:.2f}")
print(f"Total VaR: ${total_var:.2f}")

Portfolio A VaR: $5691.55
Portfolio B VaR: $4531.82
Portfolio C VaR: $3837.72
Total VaR: $13704.72


In [182]:
# Get the list of stocks in each portfolio
portfolio_A = portfolio[portfolio['Portfolio'] == 'A']['Stock'].tolist()
portfolio_B = portfolio[portfolio['Portfolio'] == 'B']['Stock'].tolist()
portfolio_C = portfolio[portfolio['Portfolio'] == 'C']['Stock'].tolist()

print(portfolio_B)

['MSFT', 'GOOGL', 'NVDA', 'JNJ', 'PG', 'MA', 'DIS', 'ADBE', 'KO', 'NFLX', 'COST', 'WFC', 'WMT', 'LLY', 'NKE', 'LIN', 'UNP', 'UPS', 'MDT', 'ORCL', 'RTX', 'AMGN', 'CAT', 'AMT', 'COP', 'AXP', 'SPGI', 'BKNG', 'ZTS', 'MDLZ', 'GILD', 'GM']


In [193]:
# Get the list of stocks in each portfolio
portfolio_A = portfolio[portfolio['Portfolio'] == 'A']['Stock'].tolist()
portfolio_B = portfolio[portfolio['Portfolio'] == 'B']['Stock'].tolist()
portfolio_C = portfolio[portfolio['Portfolio'] == 'C']['Stock'].tolist()

# Get the number of holdings for each stock in each portfolio
holdings_A = portfolio[portfolio['Portfolio'] == 'A']['Holding'].tolist()
holdings_B = portfolio[portfolio['Portfolio'] == 'B']['Holding'].tolist()
holdings_C = portfolio[portfolio['Portfolio'] == 'C']['Holding'].tolist()

# Get the daily prices for the stocks in each portfolio
portfolio_A_prices = prices[portfolio_A].values
portfolio_B_prices = prices[portfolio_B].values
portfolio_C_prices = prices[portfolio_C].values

# Calculate the daily returns for each portfolio
portfolio_A_returns = np.diff(np.log(portfolio_A_prices), axis=0)
portfolio_B_returns = np.diff(np.log(portfolio_B_prices), axis=0)
portfolio_C_returns = np.diff(np.log(portfolio_C_prices), axis=0)

# Calculate the covariance matrix for each portfolio using an AR(1) model
rho = 0.94  # Autoregression coefficient
sigma2_A = np.var(portfolio_A_returns, axis=0, ddof=1)
sigma2_B = np.var(portfolio_B_returns, axis=0, ddof=1)
sigma2_C = np.var(portfolio_C_returns, axis=0, ddof=1)
cov_A = rho * sigma2_A[:, None] * sigma2_A[None, :]
cov_B = rho * sigma2_B[:, None] * sigma2_B[None, :]
cov_C = rho * sigma2_C[:, None] * sigma2_C[None, :]

# Calculate the total covariance matrix using an AR(1) model
total_returns = np.concatenate((portfolio_A_returns, portfolio_B_returns, portfolio_C_returns), axis=1)
sigma2_total = np.var(total_returns, axis=0, ddof=1)
total_cov = rho * sigma2_total[:, None] * sigma2_total[None, :]

# Calculate the portfolio values for each portfolio
portfolio_A_values = portfolio_A_prices[-1,:] * holdings_A
portfolio_B_values = portfolio_B_prices[-1,:] * holdings_B
portfolio_C_values = portfolio_C_prices[-1,:] * holdings_C


# Calculate the total portfolio value
total_portfolio_value = np.sum(portfolio_A_values) + np.sum(portfolio_B_values) + np.sum(portfolio_C_values)

# Calculate the VaR for each portfolio and the total VaR
confidence_level = 0.95
z_score = norm.ppf(confidence_level)
portfolio_A_var = z_score * np.sqrt(np.dot(portfolio_A_values, np.dot(cov_A, portfolio_A_values)))
portfolio_B_var = z_score * np.sqrt(np.dot(portfolio_B_values, np.dot(cov_B, portfolio_B_values)))
portfolio_C_var = z_score * np.sqrt(np.dot(portfolio_C_values, np.dot(cov_C, portfolio_C_values)))
total_var = z_score * np.sqrt(np.dot(np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)), np.dot(total_cov, np.concatenate((portfolio_A_values, portfolio_B_values, portfolio_C_values)))))

print(f"Portfolio A VaR: ${portfolio_A_var}")
print(f"Portfolio B VaR: ${portfolio_B_var}")
print(f"Portfolio C VaR: ${portfolio_C_var}")
print(f"Portfolio total VaR: ${total_var}")

Portfolio A VaR: $259.31091393274534
Portfolio B VaR: $220.67137374670634
Portfolio C VaR: $172.94982943584452
Portfolio total VaR: $652.9321171152961


In [131]:
portfolio

,Portfolio,Stock,Holding
0,A,AAPL,58
1,A,TSLA,10
2,A,JPM,64
3,A,HD,27
4,A,BAC,209
...,...,...,...
94,C,F,397
95,C,LRCX,14
96,C,MO,197
97,C,LMT,27
